In [74]:
import geopandas as gpd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [81]:
# Load the shapefile
shapefile_path = "acapulco_coast_inland_training_data.shp"
gdf = gpd.read_file(shapefile_path)

gdf.columns

Index(['geometry_t', 'NDWI2', 'B11', 'B12', 'NDBSI', 'NDWI', 'NDVI', 'B2',
       'B3', 'B4', 'B8', 'BUI', 'NDBI', 'EVI', 'BAEI', 'status', 'geometry'],
      dtype='object')

In [84]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    return df[indices_to_keep].astype(np.float64)

In [89]:
# Extract the columns
all_vars = ['NDWI2', 'B11', 'B12', 'NDBSI', 'NDWI', 'NDVI', 'B2',
       'B3', 'B4', 'B8', 'BUI', 'NDBI', 'EVI', 'BAEI', 'status']
variables = ['NDWI2', 'B11', 'B12', 'NDBSI', 'NDWI', 'NDVI', 'B2',
       'B3', 'B4', 'B8', 'BUI', 'NDBI', 'EVI', 'BAEI']
label = 'status'

XX = clean_dataset(gdf[all_vars])
# Split the dataset into X (features) and y (target)



(10803, 14)
(10803,)
(4631, 14)
(4631,)


In [141]:
item_counts = XX[label].value_counts()
print(item_counts)

0.0    8162
1.0    6026
2.0    1246
Name: status, dtype: int64


In [142]:
random = XX[XX[label] != 0.0].sample(n=6000)
print(random.shape)

(6000, 15)


In [143]:
others = XX[XX[label] != 0.0]
print(others.shape)

(7272, 15)


In [144]:
balanced = pd.concat([random, others]).sample(frac = 1)
balanced.head(15)

,NDWI2,B11,B12,NDBSI,NDWI,NDVI,B2,B3,B4,B8,BUI,NDBI,EVI,BAEI,status
1281,0.063373,0.25023,0.2101,0.052265,-0.169182,0.118591,0.147475,0.165398,0.180795,0.237566,-0.104189,0.024699,0.106360,1.157350,1.0
5520,0.055032,0.29570,0.2490,0.053238,-0.184208,0.151236,0.169479,0.187667,0.199632,0.274159,-0.128130,0.036352,0.151527,1.038189,1.0
10699,0.113339,0.32740,0.2551,0.057417,-0.248608,0.174928,0.161400,0.201000,0.242000,0.344400,-0.209288,-0.025305,0.180534,0.990517,0.0
13858,0.402368,0.21570,0.1300,-0.129880,-0.538135,0.624034,0.063600,0.092400,0.075400,0.325700,-0.808182,-0.184148,0.480939,1.173513,0.0
13457,0.243301,0.20340,0.1496,-0.051087,-0.525114,0.530857,0.048600,0.067600,0.078300,0.234300,-0.710231,-0.065000,0.361397,1.308268,0.0
3892,0.187914,0.21210,0.1593,-0.002743,-0.315134,0.298999,0.106600,0.119500,0.123400,0.228715,-0.326631,-0.040410,0.224775,1.286393,1.0
11806,-0.091373,0.32510,0.2929,0.125084,-0.145063,0.097624,0.155000,0.177400,0.186000,0.242100,0.053178,0.140145,0.102223,0.977091,0.0
2332,-0.013807,0.27930,0.2388,0.091456,-0.099726,0.051474,0.171000,0.191400,0.214600,0.237000,0.038849,0.083192,0.042098,1.102050,2.0
5740,0.172045,0.27560,0.2061,0.009654,-0.329782,0.344488,0.120600,0.145800,0.137600,0.276400,-0.361235,-0.014220,0.293531,1.049165,1.0
15297,0.184936,0.23940,0.1724,0.037929,-0.323858,0.200000,0.104303,0.124467,0.164588,0.261200,-0.228929,-0.040114,0.151462,1.243999,0.0


In [145]:
X = balanced[variables]
## X.replace([np.inf, -np.inf], np.nan, inplace=True)
y = balanced[label].astype(int)
## print(X.shape)
## print(y.shape)
# Split the data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(9290, 14)
(9290,)
(3982, 14)
(3982,)


In [191]:
# Determine the input shape based on the number of features in X
input_shape = (X_train.shape[1],)

# Determine the number of classes for multi-class classification
num_classes = 3  # Adjust this based on the number of classes in your dataset

# Scale the input data to the range [-1, 1]
scaler = MinMaxScaler(feature_range=(-1, 1))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)


# Encode the target data using scikit-learn's LabelEncoder and OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y_train)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_train_encoded = onehot_encoder.fit_transform(integer_encoded)


# Create a Sequential model
model = Sequential()

# Add the input layer
model.add(Dense(128, input_shape=input_shape, activation='tanh'))

# Add a hidden layer
model.add(Dense(64, activation='relu'))

# Add the output layer with softmax activation for multi-class classification
model.add(Dense(num_classes, activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=0.01)

# Compile the model
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])


In [192]:
# Train the model for 10 epochs
history = model.fit(X_train_scaled, y_train_encoded, epochs=70, validation_split=0.2, verbose=1)

Epoch 1/70
233/233 [==============================] - 1s 5ms/step - loss: 0.6944 - categorical_accuracy: 0.7119 - val_loss: 0.6041 - val_categorical_accuracy: 0.7513
Epoch 2/70
233/233 [==============================] - 1s 5ms/step - loss: 0.6236 - categorical_accuracy: 0.7554 - val_loss: 0.5944 - val_categorical_accuracy: 0.7664
Epoch 3/70
233/233 [==============================] - 1s 5ms/step - loss: 0.5932 - categorical_accuracy: 0.7659 - val_loss: 0.5858 - val_categorical_accuracy: 0.7659
Epoch 4/70
233/233 [==============================] - 1s 5ms/step - loss: 0.5765 - categorical_accuracy: 0.7756 - val_loss: 0.5659 - val_categorical_accuracy: 0.7799
Epoch 5/70
233/233 [==============================] - 1s 5ms/step - loss: 0.5696 - categorical_accuracy: 0.7777 - val_loss: 0.5651 - val_categorical_accuracy: 0.7777
Epoch 6/70
233/233 [==============================] - 1s 5ms/step - loss: 0.5635 - categorical_accuracy: 0.7734 - val_loss: 0.5541 - val_categorical_accuracy: 0.7890
Epoc

In [193]:
X_test_scaled = scaler.fit_transform(X_test)
predictions = model.predict(X_test_scaled)

In [194]:
preds = np.argmax(predictions, axis=1)
label_encoder_test = LabelEncoder()
integer_encoded_test = label_encoder_test.fit_transform(y_test)
onehot_encoder_test = OneHotEncoder(sparse=False)
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
y_test_encoded = onehot_encoder_test.fit_transform(integer_encoded_test)
real = np.argmax(y_test_encoded, axis=1)

In [195]:
accuracy_score(real, preds)

0.6087393269713711

In [117]:
# Load the "coyuca.shp" dataset
coyuca_shapefile_path = "coyuca_benitez_characterized_data_python.shp"
coyuca_gdf = gpd.read_file(coyuca_shapefile_path)
coyuca_gdf.head()

,geometry_t,NDWI2,B11,B12,NDBSI,NDWI,NDVI,B2,B3,B4,B8,BUI,NDBI,EVI,BAEI,geometry
0,Polygon,-0.017386,0.29500,0.26040,0.066265,-0.190066,0.175899,0.164800,0.1724,0.179600,0.2561,-0.114516,0.079597,0.156730,1.028388,"POLYGON ((-100.08878 17.00507, -100.08876 17.0..."
1,Polygon,0.005984,0.30440,0.26125,0.074881,-0.160423,0.109133,0.168000,0.1899,0.210100,0.2617,-0.049363,0.061202,0.100706,1.025532,"POLYGON ((-100.08956 17.00809, -100.08951 17.0..."
2,Polygon,-0.018729,0.31710,0.28385,0.093566,-0.170097,0.105118,0.171700,0.1906,0.215800,0.2771,-0.031785,0.070035,0.100705,1.007520,"POLYGON ((-100.08833 17.00838, -100.08830 17.0..."
3,Polygon,-0.033087,0.31360,0.28130,0.096269,-0.168247,0.115432,0.172400,0.1873,0.212500,0.2649,-0.025750,0.085506,0.108976,1.012070,"POLYGON ((-100.08776 17.00824, -100.08769 17.0..."
4,Polygon,-0.007140,0.28505,0.25670,0.081350,-0.194234,0.136438,0.147892,0.1705,0.191399,0.2545,-0.071274,0.065672,0.116804,1.067725,"POLYGON ((-100.09087 17.00842, -100.09086 17.0..."


In [121]:
coyuca_clean = clean_dataset(coyuca_gdf[variables])
# Split the dataset into X (features) and y (target)
coyuca_X = coyuca_clean[variables]

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [196]:
coyuca_X_scaled = scaler.fit_transform(coyuca_X)
coyuca_preds = model.predict(coyuca_X_scaled)

In [197]:
## coyuca_gdf.drop('damage_predictions')
coyuca_gdf['damage_predictions'] =  np.argmax(coyuca_preds, axis=1)

In [200]:
coyuca_gdf.head(10)

,geometry_t,NDWI2,B11,B12,NDBSI,NDWI,NDVI,B2,B3,B4,B8,BUI,NDBI,EVI,BAEI,geometry,damage_predictions
0,Polygon,-0.017386,0.29500,0.260400,0.066265,-0.190066,0.175899,0.164800,0.172400,0.179600,0.256100,-0.114516,0.079597,0.156730,1.028388,"POLYGON ((-100.08878 17.00507, -100.08876 17.0...",0
1,Polygon,0.005984,0.30440,0.261250,0.074881,-0.160423,0.109133,0.168000,0.189900,0.210100,0.261700,-0.049363,0.061202,0.100706,1.025532,"POLYGON ((-100.08956 17.00809, -100.08951 17.0...",0
2,Polygon,-0.018729,0.31710,0.283850,0.093566,-0.170097,0.105118,0.171700,0.190600,0.215800,0.277100,-0.031785,0.070035,0.100705,1.007520,"POLYGON ((-100.08833 17.00838, -100.08830 17.0...",0
3,Polygon,-0.033087,0.31360,0.281300,0.096269,-0.168247,0.115432,0.172400,0.187300,0.212500,0.264900,-0.025750,0.085506,0.108976,1.012070,"POLYGON ((-100.08776 17.00824, -100.08769 17.0...",0
4,Polygon,-0.007140,0.28505,0.256700,0.081350,-0.194234,0.136438,0.147892,0.170500,0.191399,0.254500,-0.071274,0.065672,0.116804,1.067725,"POLYGON ((-100.09087 17.00842, -100.09086 17.0...",0
5,Polygon,-0.008168,0.30240,0.268250,0.078712,-0.163724,0.123712,0.166900,0.186100,0.202242,0.261062,-0.054990,0.066873,0.118466,1.033876,"POLYGON ((-100.08847 17.00419, -100.08847 17.0...",0
6,Polygon,0.045926,0.29020,0.245100,0.045304,-0.243859,0.220070,0.146000,0.164200,0.171250,0.273300,-0.189243,0.030522,0.207302,1.025815,"POLYGON ((-100.08249 17.00776, -100.08238 17.0...",0
7,Polygon,0.062066,0.28575,0.243750,0.039718,-0.234536,0.224368,0.147000,0.166700,0.175300,0.273650,-0.189568,0.024028,0.200320,1.047316,"POLYGON ((-100.08367 17.00940, -100.08352 17.0...",0
8,Polygon,0.081014,0.27865,0.225021,0.051318,-0.290301,0.262498,0.125155,0.145768,0.152800,0.260465,-0.243606,0.023073,0.215226,1.069699,"POLYGON ((-100.09017 17.00354, -100.09016 17.0...",0
9,Polygon,0.001246,0.29415,0.253650,0.088534,-0.194988,0.123464,0.151854,0.165232,0.192945,0.254800,-0.054423,0.074342,0.110088,1.056240,"POLYGON ((-100.08981 17.00719, -100.08980 17.0...",0


In [199]:
# coyuca_gdf['damage_predictions'].value_counts()

In [182]:
coyuca_gdf.to_file("coyuca_with_predictions_2.shp")

<ipython-input-182-5d91ab0abf2d>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  coyuca_gdf.to_file("coyuca_with_predictions_2.shp")
